## Prompt Classification Notebook

This notebook is meant to test how well the chatbot can perform the classification part of the pipeline. <br>
DISCLAIMER: We have about `$`5.00 in free API credits and are charged by how many words are inputted and output. `$`5.00 is a lot of words, probably a thousands if not hundreds of thousands so we should be okay. But preferably look to query the chatbot at an upperbound of about ~500 queries. 

### Insert prompts

Insert prompts into `PERSONA` and `CLASSIFICATION_PROMPT` variables in the below cells. `PERSONA` will provide an identity for the chatbot, an example of it is prefilled below. The `CLASSIFICATION_PROMPT` will be prepended to every user prompt that the user queries the chatbot with. It will have to be somewhat heuristic but too general at the same time, an example is prefilled below. 
<br> <br>
The classification prompt var is represented as a list so you can try many prompts and compare their outputs in the last step. Persona will not be that important/ impactful on the final result so we can change it sparingly. 

`TEMPERATURE` is defined as "The sampling temperature, between 0 and 1. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic." according to GPT documentation. Play with this number to see what temperature gives us the best results, I predict we want values closer to 0 since we don't want the chatbot messing around with users queries but feel free to experiment. 

In [5]:
PERSONA: str = '''
You are tasked with answering user queries for a hotel booking website. Your name is Nexus.
'''

CLASSIFICATION_PROMPT: list = [
'''Given the users prompt below classify what they are talking about into the following 4 categories: account query, hotel recommendation query, conversing, misc. 

If they are asking about something related to their account respond with:
{ type: ACC_QUERY }

If about adding a hotel to their watchlist, booking, or getting recommendations respond with:
{ type: HOTEL_RECC }

If they are continuing a conversation with you respond with:
{ type: CONVERSING }

If they are asking about something unrelated to the above respond with:
{ type: MISC }

Answer with these formats strictly and do not waver from it. If you are unsure about how to respond then respond with { type: MISC }.
''',

]

TEMPERATURE: float = 0.2 # placeholder

In [15]:
from openai import OpenAI

client = OpenAI()

def chat(prompt: str):
    global PERSONA, CLASSIFICATION_PROMPT, TEMPERATURE, client
    responses = []
    
    for preamble in CLASSIFICATION_PROMPT:
        messages = [{"role": "system", "content": PERSONA}]
        prompt = preamble + prompt
        messages.append({"role": "user", "content": prompt})
        
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=TEMPERATURE,
        )

        responses.append(completion.choices[0].message.content)

    return responses

In [20]:
def main():
    user_prompt = ""
    while (user_prompt != "quit"):
        user_prompt = input("Enter a prompt: ")
        print("")
        for answer in chat(user_prompt):
            print(answer)
    print("Chat ended")

main()

Enter a prompt:  What hotels can I get for under $100?



{ type: HOTEL_RECC }
